## Load Mojo Kernels

In [1]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [32]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [12]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.0034156449837610126
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.13251558190677315
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.0074024859350174665
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.11501298600342125


## Different MatMul Algorithms

In [2]:
from max.driver import Accelerator, accelerator_count, Tensor
import torch
M, K, N = 8, 8, 16 # debug
M, K, N = 4096, 6144, 2048 # real-world scale
device = Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).detach().cpu().numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [3]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    print('building cuda graph for', algorithm)
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    print('loading cuda graph...')
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
graph =  build_graph(session, "block_tiling_vectorized") # Change this to test a different algorithm
mojo_result = graph.execute(A, B)[0].to_numpy()
print("test run:\n", mojo_result, end="\n\n")
print("reference:\n", torch_result)

building cuda graph for block_tiling_vectorized
loading cuda graph...
test run:
 [[ -86.61501   -121.50275     63.004314  ...    3.512982   -70.89408
    28.695242 ]
 [  72.69593    187.93842     68.155365  ...    1.6966262   62.040924
    20.944004 ]
 [ -13.590815   -87.69274     38.643326  ...   -2.9510095  112.51922
    25.180553 ]
 ...
 [   2.7398      55.865616   -69.402405  ...  -26.78948     97.77807
   -77.70701  ]
 [ -83.45623     50.48154     26.232115  ... -127.44773     37.43157
   -55.39797  ]
 [ 218.51842     40.81186     99.22724   ...  -22.446737  -126.17001
    -4.45207  ]]

reference:
 [[ -86.6151    -121.50252     63.0043    ...    3.5128212  -70.89402
    28.695244 ]
 [  72.69581    187.93828     68.155365  ...    1.6965733   62.04103
    20.943975 ]
 [ -13.590836   -87.692764    38.64338   ...   -2.9511154  112.51915
    25.180483 ]
 ...
 [   2.7399445   55.865482   -69.40237   ...  -26.789562    97.77801
   -77.70683  ]
 [ -83.45625     50.481606    26.232098  ...

Verify kernel results:

In [9]:
if not numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.005):
    for row in range(torch_result.shape[0]):
        if numpy.allclose(torch_result[row], mojo_result[row], rtol=0, atol=0.005): continue
        print('mismatch row:', row)
else:
    print('all matched!')

all matched!


The tiled kernels can be visualized using the [matmul_visualization.mojo](./matmul_visualization.mojo) and `matmul_visualization_gui/*` scripts.

Here are some example screenshots:
![](./block_tiled_matrix_multiplication.png) 
![](./tensor_core_matmul_kernel.png)

Run a complete benchmark for different algorithms:

In [4]:
import time
for algo in [naive", "coalescing", "tiled", "tiled_register", "block_tiling", "block_tiling_vectorized", "tensor_core_matmul"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        torch.cuda.synchronize()
        begin = time.perf_counter()
        torch_result = torch_A @ torch_B
        torch.cuda.synchronize()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        torch.cuda.synchronize()
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)
        torch.cuda.synchronize()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result[0].to_numpy(), torch_result.cpu().numpy(), rtol=0, atol=0.005)
    print(algo, record)

SyntaxError: unterminated string literal (detected at line 2) (1676680648.py, line 2)

## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul